In [31]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

### Loading Data

In [32]:
data = np.load('processed_eeg_data.npz')
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']

### Calculating Hurst Scores

In [33]:
import hurst
from hurst import compute_Hc



In [44]:
df = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
print(df.shape)
print(X_train.shape)

hurst_train_exp = np.empty((X_train.shape[0], X_train.shape[1]))
hurst_train_const = np.empty((X_train.shape[0], X_train.shape[1]))

for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        hurst_train_exp[i, j], hurst_train_const[i,j], _ = compute_Hc(X_train[i, j], kind='change', simplified=True)


    
    if i % 100 == 0:
        print(f"Processed {i} samples")


(8282, 5888)
(8282, 23, 256)
Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples
Processed 700 samples
Processed 800 samples
Processed 900 samples
Processed 1000 samples
Processed 1100 samples
Processed 1200 samples
Processed 1300 samples
Processed 1400 samples
Processed 1500 samples
Processed 1600 samples
Processed 1700 samples
Processed 1800 samples
Processed 1900 samples
Processed 2000 samples
Processed 2100 samples
Processed 2200 samples
Processed 2300 samples
Processed 2400 samples
Processed 2500 samples
Processed 2600 samples
Processed 2700 samples
Processed 2800 samples
Processed 2900 samples
Processed 3000 samples
Processed 3100 samples
Processed 3200 samples
Processed 3300 samples
Processed 3400 samples
Processed 3500 samples
Processed 3600 samples
Processed 3700 samples
Processed 3800 samples
Processed 3900 samples
Processed 4000 samples
Processed 4100 samples
Processed 4200 sa

In [47]:
hurst_train_exp.shape, hurst_train_const.shape

((8282, 23), (8282, 23))

### SVM Modeling

In [ ]:
# Support Vector Classfier (a type of SVM), uses linear decision boundary
clf = svm.SVC(kernel="linear",probability=True)

# Flatten the datasets to 2D as SVC expects 2D array input
X_train = X_train.reshape(X_train.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0], -1)

# Fits SVM on the training data, 
# I don't know why they are doing this as the next line re-fits the model.
clf.fit(X_train, y_train)

# k-fold cross validation with 10 folds
y_pred = cross_val_predict(clf, X_val, y_val, cv=10)
print("All features are included\n", classification_report(y_val, y_pred))

C:\Users\andre\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
